# A Network Simplex Energy Systems Model

What if we could re-formulate MILP energy systems models as a minimum-cost-flow problem? This would allow us to use super-efficient simplex methods to solve for energy flow, basically creating the minimum-cost sankey diagram through time.

**Advantages:**
- model using plant-based data
- flexible region-sector-time definition
- min-cost simplex problem

**Trade-offs:**
- integer-value capacity-expansions

**Questions:**
- emissions and other constraints?
- operating models?
- storage?

![alternative text](netflow_esys_model.png)

### Let's demonstrate the capacity expansion mechanism first

**Problem Formulation:**
Let's find the minimum cost flow for a graph, etc. etc.

![alternative text](simple_toy.png)

In [1]:
from pydantic import BaseModel
from itertools import chain, product

**Define some plants / technologies**

In [182]:
from typing import Optional

In [183]:
class Plant(BaseModel):
    id:str                       # id
    remaining_life: int          # years
    output_capacity: int                # kW
    io_ratio: float              # efficiency
    capex: int                   # $
    opex: int                    # $/KWh
    commodity_input: str         # INPUT commodity
    commodity_output: str        # OUTPUT commodity
    availability_year: int       # first year
    inp_nodes: Optional[list] = None
    outp_nodes: Optional[list] = None
    capexp_node: Optional[str] = None
    active_yrtimes: Optional[list] = None
    input_capacity: int = None

In [184]:
initial_plants = [
    Plant(
        id='plant-existing-0',
        remaining_life=5,
        output_capacity = 2000,
        io_ratio = 0.8,
        capex = 0,
        opex = 1,
        commodity_input = "INPUT",
        commodity_output = "OUTPUT",
        availability_year = 0,
        input_capacity = int(2000/0.8),
    ),
    Plant(
        id='plant-existing-1',
        remaining_life=15,
        output_capacity = 5000,
        io_ratio = 0.6,
        capex = 0,
        opex=2,
        commodity_input = "INPUT",
        commodity_output = "OUTPUT",
        availability_year = 0,
        input_capacity = int(5000/0.6),
    )
]

In [185]:
new_plants = [
    [
        Plant(
            id=f'plant-new-{year}-0',
            remaining_life=10,
            output_capacity=1000,
            io_ratio = 0.8-0.2*(year/30),
            capex = 1500*1000,
            opex = 1,
            availability_year = year,
            commodity_input = "INPUT",
            commodity_output = "OUTPUT",
            input_capacity = int(1000/( 0.8-0.2*(year/30))),
        ),
        Plant(
            id=f'plant-new-{year}-1',
            remaining_life=20,
            output_capacity=2000,
            io_ratio =  0.6-0.1*(year/30),
            capex = 1000*2000,
            opex = 2,
            availability_year = year,
            commodity_input = "INPUT",
            commodity_output = "OUTPUT",
            input_capacity = int(2000/(0.6-0.1*(year/30))),
        ),
    ]
    for year in range(30)
]
all_plants = initial_plants + list(chain(*new_plants))

**Set some demands**

In [186]:
times = range(12)
years = range(30)
peak_demands = {yr:6000+500*yr for yr in years}
yr_time_demand = {f'{yr}-{t}':int(0.5*peak_demands[yr]+0.5*peak_demands[yr]*(1-(abs((t-6)/6)))) for yr, t in product(years, times)}
commodities = ["INPUT","OUTPUT"]

### BUILD THE GRAPH

In [7]:
import networkx as nx

In [207]:
G = nx.DiGraph()

**Add all the Nodes**

In [208]:
# commodity buses
commodity_nodes = ['-'.join([commodity,yr_time]) for commodity, yr_time in product(commodities, yr_time_demand.keys())]

In [209]:
G.add_nodes_from(commodity_nodes)

In [210]:
# add existing plants and new plants
for plant in all_plants:
    # add input nodes
    plant.active_yrtimes = [[yr,t] for yr, t in product(range(plant.availability_year, min(30,plant.availability_year+plant.remaining_life)), times)]
    inp_nodes = [f'{plant.id}-{yr}-{t}-{plant.commodity_input}' for yr, t in product(range(plant.availability_year, min(30,plant.availability_year+plant.remaining_life)), times)]
    # add output nodes
    outp_nodes = [f'{plant.id}-{yr}-{t}-{plant.commodity_output}' for yr, t in product(range(plant.availability_year, min(30,plant.availability_year+plant.remaining_life)), times)]
    # add cap_exp_node
    add_nodes = inp_nodes + outp_nodes + [f'{plant.id}-capexp']
    plant.inp_nodes = inp_nodes
    plant.outp_nodes = outp_nodes
    plant.capexp_node = f'{plant.id}-capexp'
    G.add_nodes_from(add_nodes)

In [211]:
# add optimisation nodes
global_demand = sum(yr_time_demand.values())
LARGE_VAL = global_demand*100
optim_nodes = [
    ('GLOBAL_DEMAND',{'demand':global_demand}),
    ('GLOBAL_SUPPLY',{'demand':-LARGE_VAL}),
    ('LOSS_SINK',{'demand':LARGE_VAL-global_demand}),
    ('CAPEXP_SOURCE',{'demand':-LARGE_VAL}),
    ('CAPEXP_SINK',{'demand':LARGE_VAL}),
]
G.add_nodes_from(optim_nodes)

In [212]:
optim_nodes

[('GLOBAL_DEMAND', {'demand': 3577420}),
 ('GLOBAL_SUPPLY', {'demand': -357742000}),
 ('LOSS_SINK', {'demand': 354164580}),
 ('CAPEXP_SOURCE', {'demand': -357742000}),
 ('CAPEXP_SINK', {'demand': 357742000})]

**Add all the edges**

*... here be dragons...*

In [213]:
SUPPLY_COST=1

In [214]:
### ENERGY FLOW
# input & output edges
input_edges = [ ('GLOBAL_SUPPLY','-'.join(['INPUT',yr_time]),{'capacity':LARGE_VAL,'cost_var':SUPPLY_COST,'cost_fixed':0}) for yr_time in yr_time_demand.keys()]
output_edges = [('-'.join(['OUTPUT',yr_time]),'GLOBAL_DEMAND',{'capacity':val, 'cost_var':0, 'cost_fixed':0}) for yr_time, val in yr_time_demand.items()]
G.add_edges_from(input_edges+output_edges)

In [215]:
# plant endges
for plant in all_plants:
    input_edges =  [('-'.join([plant.commodity_input,str(yr),str(t)]), f'{plant.id}-{yr}-{t}-{plant.commodity_input}', {'capacity':plant.input_capacity, 'cost_var':0, 'cost_fixed':0}) for yr,t in plant.active_yrtimes]# inp-commod to inp-plant
    bridge_edges = [(f'{plant.id}-{yr}-{t}-{plant.commodity_input}',f'{plant.id}-{yr}-{t}-{plant.commodity_output}', {'capacity':plant.output_capacity-1, 'cost_var':0, 'cost_fixed':0}) for yr,t in plant.active_yrtimes]
    output_edges = [(f'{plant.id}-{yr}-{t}-{plant.commodity_output}','-'.join([plant.commodity_output,str(yr),str(t)]), {'capacity':plant.output_capacity, 'cost_var':plant.opex, 'cost_fixed':0}) for yr,t in plant.active_yrtimes]
    loss_edges = [(f'{plant.id}-{yr}-{t}-{plant.commodity_input}','LOSS_SINK', {'capacity':plant.input_capacity-plant.output_capacity, 'cost_var':0, 'cost_fixed':0}) for yr,t in plant.active_yrtimes]
    capexp_minus_edges = [(f'{plant.id}-{yr}-{t}-{plant.commodity_input}','CAPEX_SINK',{'capacity':1, 'cost_var':0, 'cost_fixed':0}) for yr,t in plant.active_yrtimes]
    capexp_plus_edges = [(plant.capexp_node,f'{plant.id}-{yr}-{t}-{plant.commodity_input}',{'capacity':1, 'cost_var':0, 'cost_fixed':0}) for yr,t in plant.active_yrtimes]
    capexp_edge = [('CAPEXP_SOURCE',plant.capexp_node,{'capacity':LARGE_VAL, 'cost_var':0, 'cost_fixed':plant.capex})]
    G.add_edges_from(input_edges+bridge_edges+output_edges+loss_edges+capexp_minus_edges+capexp_plus_edges+capexp_edge)

In [216]:
# add bypasses
supply_bypass = ('GLOBAL_SUPPLY','LOSS_SINK',{'capacity':LARGE_VAL,'cost_var':LARGE_VAL*1e9,'cost_fixed':0})
capexp_bypass = ('CAPEXP_SOURCE','CAPEXP_SINK',{'capacity':LARGE_VAL,'cost_var':0,'cost_fixed':0})
G.add_edges_from([supply_bypass, capexp_bypass])

In [217]:
from my_simplex import my_network_simplex

In [218]:
cost, outflows = my_network_simplex(G, demand="demand", capacity="capacity", weight_var="cost_var", weight_fixed="cost_fixed")

In [219]:
inflows = {kk:{} for kk in flows.keys()}

In [220]:
for kk1,vv in flows.items():
    for kk2, val in vv.items():
        inflows[kk2][kk1] = val

In [221]:
outflows['plant-existing-1-0-0-INPUT'], inflows['plant-existing-1-0-0-INPUT']

({'plant-existing-1-0-0-OUTPUT': 3000, 'LOSS_SINK': 3333, 'CAPEX_SINK': 0},
 {'INPUT-0-0': 3000, 'plant-existing-1-capexp': 0})

In [222]:
inflows['OUTPUT-0-0'], outflows['OUTPUT-0-0']

({'plant-existing-0-0-0-OUTPUT': 0,
  'plant-existing-1-0-0-OUTPUT': 3000,
  'plant-new-0-0-0-0-OUTPUT': 0,
  'plant-new-0-1-0-0-OUTPUT': 0},
 {'GLOBAL_DEMAND': 3000})

In [223]:
inflows['plant-existing-1-0-0-INPUT'],outflows['plant-existing-1-0-0-INPUT']

({'INPUT-0-0': 3000, 'plant-existing-1-capexp': 0},
 {'plant-existing-1-0-0-OUTPUT': 3000, 'LOSS_SINK': 3333, 'CAPEX_SINK': 0})

In [206]:
inflows['CAPEXP_SINK']

{'CAPEXP_SOURCE': 357742000}